In [23]:
import selenium
from selenium import webdriver
import pandas as pd
import re

In [24]:
browser=webdriver.Chrome('C:/Users/Administrator/chromedriver.exe')

In [25]:
browser.get('https://www.zomato.com/bangalore/great-food-no-bull')

In [11]:
restaurants=browser.find_elements_by_css_selector('div.col-s-8.col-l-1by3')
print(len(restaurants))

141


In [26]:
def get_reviews(var,selector,attribute):
    try:
        tag=var.find_element_by_css_selector(selector)
        tag=tag.get_attribute(attribute)
    except: 
        tag=0
    return tag

In [27]:
url_info=pd.DataFrame(columns=['Restaurant Name','Restaurant ID','URL'])

In [14]:
for restaurant in restaurants:
    
    name=get_reviews(restaurant,'div.res_title.zblack.bold','title')
    name=name.lstrip()
    
    r_id=get_reviews(restaurant,'div.relative.top-res-box.entity-ads-snippet-track','data-res-id')
    
    url=get_reviews(restaurant,'a','href')
  
    rest1={'Restaurant Name':name,'Restaurant ID':r_id,'URL':url}
    url_info=url_info.append(rest1,ignore_index=True)
url_info=url_info.head(55)
url_info.head()

,Restaurant Name,Restaurant ID,URL
0,Burma Burma,18625420,https://www.zomato.com/bangalore/burma-burma-i...
1,The Pizza Bakery,18603822,https://www.zomato.com/bangalore/the-pizza-bak...
2,Nasi And Mee,18476467,https://www.zomato.com/bangalore/nasi-and-mee-...
3,Prost Brew Pub,54503,https://www.zomato.com/bangalore/prost-brew-pu...
4,Roots,18270163,https://www.zomato.com/bangalore/roots-koraman...


In [28]:
restaurant_info=pd.DataFrame(columns=['Restaurant Name','Restaurant ID','User Name','User ID','Rating'])

In [29]:
c=0
for i in range(len(url_info)):
    c=c+1
    print(c)
    browser.get(url_info['URL'].iloc[i])
    try:
        r_id = url_info['Restaurant ID'].iloc[i]
        name=browser.find_element_by_css_selector('h1.res-name.left.mb0')
        name=name.find_element_by_css_selector('a')
        name=name.get_attribute('innerHTML').lstrip()
        name=re.sub('amp;','',name)
    except:
        r_id=None
        continue
    if name in url_info['Restaurant Name'].iloc[i]:
        
        while True:
            try:
                reviews=browser.find_elements_by_css_selector('div.ui.segments.res-review-body.res-review.clearfix.js-activity-root.mbti.item-to-hide-parent.stupendousact') 
                for review in reviews:
                    try:
                        user_name=review.find_element_by_css_selector('div.header.nowrap.ui.left')
                        user_name=user_name.find_element_by_css_selector('a')
                        user_name=user_name.get_attribute('innerHTML')
                        user_name=user_name.strip()
                        user_id=review.find_element_by_css_selector('div.header.nowrap.ui.left')
                        user_id=user_id.find_element_by_css_selector('a')
                        user_id=user_id.get_attribute('data-entity_id')
                    except:
                        user_name=None
                        user_id=None

                    try:
                        rating=review.find_element_by_css_selector('div.ttupper.fs12px.left.bold.zdhl2.tooltip')
                        rating=rating.get_attribute('aria-label')
                    except:
                        rating=None
                    rating=re.sub('Rated','',rating)

                    rest={'Restaurant Name':name,'Restaurant ID':r_id,'User Name':user_name,'User ID':user_id,'Rating':rating}
                    restaurant_info=restaurant_info.append(rest,ignore_index=True)
                browser.find_element_by_css_selector('div.ui.segment.clearfix.zs-load-more.res-page-load-more').click()
            except:
                break

In [30]:
restaurant_info.to_csv("Reviews.csv",index=False)
restaurant_info.shape

(0, 5)

In [9]:
#for i in range(20):
#    for j in range(20):
#        if restaurant_info['Restaurant Name'].iloc[j]==url_info['Restaurant Name'].iloc[i]:
#            restaurant_info['Restaurant ID']=url_info['Restaurant ID']
#restaurant_info

In [10]:
# url_info.index = url_info['Restaurant Name']
# res_map = url_info['Restaurant ID'].to_dict()

# restaurant_info['Restaurant ID'] = restaurant_info['Restaurant Name'].replace(res_map)
# restaurant_info

In [11]:
#number of distinct restaurants present in the data
restaurant_info['Restaurant Name'].nunique()

0

# MODEL BUILDING

In [12]:
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics.pairwise import cosine_similarity
from surprise import KNNWithZScore
from surprise import SVD, dataset,Reader
import pandas as pd
import numpy as np

In [13]:
review=pd.read_csv('C:/Users/Administrator/Downloads/Reviews.csv')
review.head()

,Restaurant Name,Restaurant ID,User Name,User ID,Rating
0,Red Rhino,18492910,Utsav Walia,49291855,2.0
1,Red Rhino,18492910,Richa Kulshrestha,2208232,4.0
2,Red Rhino,18492910,Ekta Nayak,27365170,5.0
3,Red Rhino,18492910,Gupteswar Prasad Sarangi,1343707,4.0
4,Red Rhino,18492910,Nikhil Agrawal,31914860,5.0


In [18]:
scaler=StandardScaler()
ui_review_matrix=review.pivot_table(columns='Restaurant ID',index='User ID',values='Rating')
null_matrix=ui_review_matrix
scaler.fit(ui_review_matrix)
# ui_review_matrix=ui_review_matrix.apply(lambda x: ((x-x.mean)/x.std()),axis=1).fillna(0)
ui_review_matrix=pd.DataFrame(scaler.transform(ui_review_matrix),columns=ui_review_matrix.columns,index=ui_review_matrix.index)
ui_review_matrix=ui_review_matrix.fillna(0)
ui_review_matrix.head()

In [ ]:
sim_matrix=pd.DataFrame(cosine_similarity(ui_review_matrix),index=ui_review_matrix.index,
                       columns=ui_review_matrix.index)
sim_matrix.head()

In [ ]:
def recommend(userid):
    neigh=sim_matrix[userid].drop(userid).sort_values(ascending=False).head(5).index
    neigh_matrix=ui_review_matrix.loc[neigh]
    user_restaraunt=null_matrix.loc[userid]
    not_gone=user_restaraunt[pd.isnull(user_restaraunt)]
    pred=neigh_matrix.mean()[not_gone.index]
    recommend=pred.sort_values(ascending=False).head().index
    data=review[review['Restaurant ID'].isin(recommend)]['Restaurant Name'].unique()
    return pd.DataFrame(data).rename(columns={0:'Restaraunt Name'})

In [ ]:
recommend(11876)